### LCEL = LangChain Expression Language
It is used to chain components together.

#### Open source models: llama3, gemma2, mistral -> GROQ api key

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_KEY"] = os.getenv("LANGCHAIN_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9B-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10db17e50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10db27b80>, model_name='Gemma2-9B-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from english to french"),
    HumanMessage(content="Hello, how are you?")
]

result = model.invoke(messages)
result

AIMessage(content="Hello, how are you?  translates to:\n\n**Bonjour, comment allez-vous ?** \n\n\nLet me know if you'd like to translate anything else! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 22, 'total_tokens': 62, 'completion_time': 0.072727273, 'prompt_time': 0.00016946, 'queue_time': 0.014337098, 'total_time': 0.072896733}, 'model_name': 'Gemma2-9B-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c1aebef1-a2aa-4e7a-b566-13fbbf0c634b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 40, 'total_tokens': 62})

In [16]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

"Hello, how are you?  translates to:\n\n**Bonjour, comment allez-vous ?** \n\n\nLet me know if you'd like to translate anything else! 😊 \n"

In [18]:
## Using LCEL, we can chain the components
chain = model|parser
chain.invoke(messages)

"Hello, how are you?  translates to: \n\n**Bonjour, comment allez-vous ?** (formal) \n\nor\n\n**Salut, ça va ?** (informal) \n\n\nLet me know if you'd like to translate anything else! 😊 \n"

In [19]:
## Prompts templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [22]:
result = prompt.invoke({"language": "french", "text": "Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [23]:
result.to_messages()

[SystemMessage(content='Translate the following into french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [26]:
chain = prompt|model|parser
chain.invoke({"language": "hindi", "text": "Hello"})

'नमस्ते \n\n\nHere are some ways to say "Hello" in Hindi:\n\n* **नमस्ते (Namaste)** - This is the most common and formal way to greet someone.\n\nLet me know if you\'d like to learn other greetings! \n'